In [2]:
import os
os.chdir(r'/home/fruitspec-lab/FruitSpec/Code/roi/fsCounter')
from MHS.F_model_training import *
from math import radians, cos, sin, asin, sqrt
from vision.misc.help_func import go_up_n_levels

os.chdir(r'/home/fruitspec-lab/FruitSpec/Code/roi/fsCounter/MHS')
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display


<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
/home/fruitspec-lab/anaconda3/envs/FsCounterGPUtest/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [56]:
use_best_study = True
csv_path = "/media/fruitspec-lab/cam175/MOTCHA/features_MOTCHA_6_8.csv"
cfg = OmegaConf.load("/home/fruitspec-lab/FruitSpec/Code/roi/fsCounter/MHS/model_config.yaml")
final_cols, drop_final = get_rel_cols(cfg)
f_df = read_f_df(cfg)
features_df = pd.read_csv(csv_path)


In [57]:
out_frame = infer_on_features(features_df, cfg, model_pkl="/home/fruitspec-lab/FruitSpec/Code/roi/fsCounter/MHS/models_3007/lasso_pipe.pkl",
                              tree_model=False)
output = out_frame
output.rename({"F_pred": "Lasso_pred"}, axis = 1, inplace = True)

out_frame = infer_on_features(features_df, cfg, model_pkl="/home/fruitspec-lab/FruitSpec/Code/roi/fsCounter/MHS/models_3007/xgb_notebook.pkl",
                              tree_model=True)
output["XGB_pred"] = out_frame["F_pred"]


In [58]:
output

,block_name,name,customer,Lasso_pred,XGB_pred
0,MEIRAVHA,R1_S1_T0,MOTCHA,481.132208,408.998657
1,MEIRAVHA,R1_S1_T1,MOTCHA,1140.369591,606.773743
2,MEIRAVHA,R1_S1_T2,MOTCHA,499.248042,452.419495
4,MEIRAVHA,R1_S1_T4,MOTCHA,626.121203,568.194397
6,MEIRAVHA,R1_S1_T6,MOTCHA,579.090267,448.943146
...,...,...,...,...,...
457,SATSUMAX,R6_S1_T19,MOTCHA,357.926044,276.429108
458,SATSUMAX,R6_S1_T20,MOTCHA,477.148897,398.846283
459,SATSUMAX,R6_S1_T21,MOTCHA,561.482831,470.005768
460,SATSUMAX,R6_S1_T22,MOTCHA,363.880586,313.120087


In [59]:
def distance(lat1, lat2, lon1, lon2):
    # convert from degrees to radians
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2

    c = 2 * asin(sqrt(a))

    # Radius of earth in kilometers. Use 3956 for miles
    r = 6371

    # calculate the result
    return (c * r) * 1000  # results in M

def get_row_length(df_jz):
    if isinstance(df_jz, str):
        df_jz = pd.read_csv(df_jz)
    df_merged = df_jz
    df_merged = df_merged.fillna(method='bfill')

    if df_merged is None: # None where is no overlap between GPS and JZ
        return 0
    first_index = min(df_merged.index)
    last_index = max(df_merged.index)
    lat1 = float(df_merged.latitude[first_index])
    lat2 = float(df_merged.latitude[last_index])
    lon1 = float(df_merged.longitude[first_index])
    lon2 = float(df_merged.longitude[last_index])

    return distance(lat1, lat2, lon1, lon2)


def get_valid_row_paths(master_folder):
    paths_list = []
    for root, dirs, files in os.walk(master_folder):
        if np.all([file in files for file in ["jaized_timestamps.csv"]]):
            row_scan_path = os.path.abspath(root)
            paths_list.append(os.path.join(row_scan_path, "jaized_timestamps.csv"))
    return paths_list

def get_full_name_from_path(path_to_row_jz):
    customer_name = os.path.basename(go_up_n_levels(path_to_row_jz,5))
    block_name = os.path.basename(go_up_n_levels(path_to_row_jz,4))
    row_name = "R" + os.path.basename(go_up_n_levels(path_to_row_jz,2)).split("_")[-1]
    scan_name = "S" + os.path.basename(go_up_n_levels(path_to_row_jz,1)).split("_")[-1]
    full_name = f"{customer_name}_{block_name}_{row_name}_{scan_name}"
    return full_name

def run_on_folder(master_folder, njobs=1):
    paths_list = get_valid_row_paths(master_folder)
    n = len(paths_list)
    if njobs > 1:
        with ProcessPoolExecutor(max_workers=njobs) as executor:
            res = list(executor.map(get_row_length, paths_list))
    else:
        res = list(map(get_row_length, paths_list))
    res_names = list(map(get_full_name_from_path, paths_list))
    return dict(zip(res_names, res))

        
master_folder = "/media/fruitspec-lab/cam175/MOTCHA"
row_lengths = run_on_folder(master_folder)


row_lengths

{'MOTCHA_MEIRAVHA_R1_S1': 129.03882156265203,
 'MOTCHA_MEIRAVHA_R2_S1': 124.84925921679138,
 'MOTCHA_MEIRAVHA_R3_S1': 124.29658414521478,
 'MOTCHA_MEIRAVHA_R4_S1': 119.30544974291938,
 'MOTCHA_MICHAL22_R1_S1': 131.48311675807213,
 'MOTCHA_MICHAL22_R10_S1': 121.29889156356643,
 'MOTCHA_MICHAL22_R2_S1': 131.05960598697297,
 'MOTCHA_MICHAL22_R3_S1': 132.47122911518073,
 'MOTCHA_MICHAL22_R4_S1': 133.10985244185943,
 'MOTCHA_SATSUMAX_R1_S1': 80.63077504146639,
 'MOTCHA_SATSUMAX_R2_S1': 80.98593205441576,
 'MOTCHA_SATSUMAX_R3_S1': 80.0056450725284,
 'MOTCHA_SATSUMAX_R4_S1': 81.01839205635869,
 'MOTCHA_SATSUMAX_R5_S1': 80.5030348717008,
 'MOTCHA_SATSUMAX_R6_S1': 80.92576898426255}

In [60]:
def extract_row_name(row_name):
    return row_name.split('_T')[0]
def extract_number_after_T(row_name):
    return int(row_name.split('_T')[1])
output['row_name'] = output.apply(lambda row: f"{row['customer']}_{row['block_name']}_{extract_row_name(row['name'])}", axis=1)
output['tree_number'] = output["name"].apply(extract_number_after_T)

In [61]:
planting_dist = {"MEIRAVHA": 2, "MICHAL22":2, 'RAUSTENB':2, "SATSUMAX":4}

In [62]:
output["planting_dist"] = output["block_name"].map(planting_dist)

In [63]:
output.round(2).to_csv(f"{csv_path[:-4]}_w_preds.csv")
output.groupby("block_name").median().round(2).to_csv(f"{csv_path[:-4]}_block_means.csv")
output.groupby("block_name").std().round(2).to_csv(f"{csv_path[:-4]}_block_stds.csv")

grouped_df = output.groupby("row_name").agg({
    "tree_number": "max",  # Count the occurrences of each group
    "Lasso_pred": [np.median, np.std],
    "XGB_pred": [np.median, np.std],
    "planting_dist": np.mean,
})

new_cols = ["count", "Lasso_pred mean", "Lasso_pred std", "XGB_pred mean", "XGB_pred std", "planting_dist"]
grouped_df.columns = new_cols

rows = [name.split("_")[2] for name in grouped_df.index]
blocks = [name.split("_")[1] for name in grouped_df.index]
grouped_df["row"] = rows
grouped_df["block_name"] = blocks
grouped_df["row_len"] = grouped_df.index.map(row_lengths)
grouped_df = grouped_df[grouped_df["row_len"] > 1]
for col in ["Lasso_pred", "XGB_pred"]:
    grouped_df[f"{col} per meter"] = grouped_df[f"{col} mean"] * grouped_df["count"] / grouped_df["row_len"]
    grouped_df[f"{col} per tree"] = grouped_df[f"{col} per meter"] * grouped_df["planting_dist"]
grouped_df.round(2).to_csv(f"{csv_path[:-4]}_row_summary.csv")
grouped_df.groupby("block_name").median().round(2).to_csv(f"{csv_path[:-4]}_block_means_per_meter.csv")
grouped_df.groupby("block_name").std().round(2).to_csv(f"{csv_path[:-4]}_block_stds_per_meter.csv")

grouped_df = output.groupby("block_name").agg({
    "name": "count",  # Count the occurrences of each group
    "Lasso_pred": [np.median, np.std],
    "XGB_pred": [np.median, np.std],
    "planting_dist": np.mean,
})

# Rename the columns for better readability
grouped_df.columns = new_cols
grouped_df.round(2).to_csv(f"{csv_path[:-4]}_summary_block.csv")